In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy 
nlp = spacy.load('en_core_web_sm')
import re
from geopy.geocoders import Nominatim
from tqdm import tqdm

C:\Users\Noa Ehrenhalt\AppData\Roaming\Python\Python38\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Noa Ehrenhalt\AppData\Roaming\Python\Python38\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\Noa Ehrenhalt\AppData\Roaming\Python\Python38\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


function to convert to name entitiy 

In [3]:
df = pd.read_csv('pos_and_neg_tweets_balanced_19_02_21.csv')

In [4]:
df.head()

,Unnamed: 0,user_name,user_location,user_description,user_created,user_followers,date,text,favorites,retweets,is_retweet,category
0,0,AARPKY,Bluegrass State,Official AARP Kentucky Twitter. Get to know us...,2009-07-25 15:14:41,3266,2021-02-18 14:12:52,@3rdRetiredTeach See Kentucky's update vaccine...,0,0,False,Positive
1,1,GoSolar01,NaN,"Get all your solar info here, including latest...",2017-12-14 06:26:39,1861,2021-02-18 05:35:56,RT @dr_hhq: Chipped - #crushcovid #gettheshot ...,0,1,True,Positive
2,2,dr_hhq,"Karachi, Pakistan",Assistant Professor #Urology #SIUT - Half Prof...,2009-08-15 06:11:55,2378,2021-02-18 05:35:37,Chipped - #crushcovid #gettheshot 🇨🇳#covid #co...,2,1,True,Positive
3,3,Nakhasi_MD,"Los Angeles, CA",Doc in Compton | Policy Adviser. LA County Dep...,2009-03-11 06:04:06,2205,2021-02-18 03:58:45,"RT @PinnacleTC_Hope: Holly Broce, president of...",0,1,True,Positive
4,4,SonLaurencio,"Kentucky, USA",Pro Executive Entrepreneur #business #marketin...,2013-04-15 02:31:40,1558,2021-02-18 03:48:33,RT @LFCHD: A COVID-19 vaccine update thread:\n...,0,1,True,Positive


In [5]:
df.shape

(36696, 12)

In [6]:
locations = df.user_location

clean the locations from irellevant charachters

In [7]:
locations = locations.replace(np.nan,'')
locations = locations.apply(lambda x: re.sub("[^a-zA-Z\s,]+", "", x))

In [8]:
df['user_location_updated'] = locations

In [9]:
tqdm.pandas()


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
def geo_locator(user_location):
    
    # initialize geolocator
    geolocator = Nominatim(user_agent='Tweet_locator')

    if user_location is not None:
        try :
            # get location
            location = geolocator.geocode(user_location, language='en')
            # get coordinates
            location_exact = geolocator.reverse(
                        [location.latitude, location.longitude], language='en')
            # get country codes
            c_code = location_exact.raw['address']['country_code']

            return c_code

        except:
            return None

    else : 
        return None


In [11]:
# apply geo locator to user-location
loc = df['user_location_updated'].progress_apply(geo_locator)
df['user-country_code'] = loc

100%|█████████████████████████████████████████████████| 36696/36696 [6:09:07<00:00,  1.66it/s]


In [12]:
# !pip install country_converter

In [13]:
import country_converter as coco

In [14]:
to_iso3_func = lambda x: coco.convert(names=x, to='iso3', not_found=None) \
                            if x is not None else x


In [15]:
# change codes to iso3 
df['user-country_code'] = df['user-country_code'].progress_apply(to_iso3_func)


100%|███████████████████████████████████████████████████| 36696/36696 [16:23<00:00, 37.31it/s]


In [16]:
to_std_func = lambda x: coco.convert(names=x, to='name_short', not_found=None) \
                            if x is not None else x

In [17]:
# create user-country column
df['user-country'] = df['user-country_code'].progress_apply(to_std_func)

100%|███████████████████████████████████████████████████| 36696/36696 [16:30<00:00, 37.04it/s]


In [18]:
df['user-country'].value_counts().sum()

15116

In [19]:
df['user-country'].unique()

array(['United States', None, 'Pakistan', 'Canada', 'United Kingdom',
       'Hungary', 'Romania', 'Italy', 'Germany', 'Kenya', "Cote d'Ivoire",
       'Luxembourg', 'Greece', 'Cyprus', 'India', 'United Arab Emirates',
       'South Korea', 'Denmark', 'France', 'China', 'Myanmar',
       'Philippines', 'Ireland', 'Paraguay', 'Brazil', 'Chile', 'Peru',
       'Thailand', 'South Africa', 'Costa Rica', 'Belgium', 'Japan',
       'Zimbabwe', 'Maldives', 'Lebanon', 'Bangladesh', 'Australia',
       'Qatar', 'New Zealand', 'Switzerland', 'Mexico', 'DR Congo',
       'Latvia', 'Mongolia', 'Colombia', 'Guatemala',
       'Trinidad and Tobago', 'Netherlands', 'Israel', 'Norway',
       'Sri Lanka', 'South Sudan', 'Nepal', 'Nigeria', 'Papua New Guinea',
       'Ghana', 'Somalia', 'Venezuela', 'Malaysia', 'Iran', 'Kosovo',
       'Finland', 'Chad', 'Dominican Republic', 'Kuwait', 'Spain',
       'Isle of Man', 'Austria', 'Cambodia', 'Cameroon', 'Kazakhstan',
       'Syria', 'Iraq', 'Portugal', 'I

In [20]:
df.to_csv("pos_and_neg_tweets_balanced_final.csv")

In [21]:
df.columns

Index(['Unnamed: 0', 'user_name', 'user_location', 'user_description',
       'user_created', 'user_followers', 'date', 'text', 'favorites',
       'retweets', 'is_retweet', 'category', 'user_location_updated',
       'user-country_code', 'user-country'],
      dtype='object')

In [22]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [23]:
df_up = df.drop_duplicates()

In [24]:
df_up.shape

(19039, 14)

In [25]:
number_of_na = df_up.isna().sum()
filter_na = number_of_na > 0
columns_with_na = number_of_na[filter_na]
columns_with_na

user_location         4598
user_description      1917
user-country_code    11163
user-country         11163
dtype: int64

In [26]:
df.to_csv("pos_and_neg_tweets_balanced_no_duplicates.csv")